In [ ]:
# ====== Section 1: Imports and Setup ======
# Standard library imports for data handling and machine learning.
import pickle
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Imports from PyTorch for model definition.
import torch.nn as nn
import torch.optim as optim

# Imports for quantization and model inspection from NNDCT (Neural Network Distiller and Compiler Tools), a toolkit for optimizing models.
import pytorch_nndct
from pytorch_nndct import Inspector

# Set batch size for data loading.
batch_size = 1

In [ ]:
# ====== Section 2: Data Loading and Preparation ======
# Load dataset from a Pickle file.
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')

# Extract specific modulation types and signal-to-noise ratios (SNR) data.
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# Generate labels for the two types of modulation.
qpsk_labels = [1] * 1000  # QPSK labeled as 1.
bpsk_labels = [0] * 1000  # BPSK labeled as 0.

# Combine data and labels from both modulation types.
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = np.array(qpsk_labels + bpsk_labels, dtype=np.int64)

# Convert combined data and labels to PyTorch tensors.
data_combined = torch.from_numpy(data_combined).float()
labels_combined = torch.from_numpy(labels_combined).long()

# Split data into training and testing sets.
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=0.2, random_state=42)

In [ ]:
# ====== Section 3: Dataset and DataLoader Definition ======
# Define a custom dataset class for loading the data.
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]
        # Normalize the sample.
        epsilon = 1e-10
        min_val = sample.min(axis=1, keepdim=True)
        max_val = sample.max(axis=1, keepdim=True)
        normalized_sample = 2 * (sample - min_val) / (max_val - min_val + epsilon) - 1
        return normalized_sample, label

# Initialize training and testing datasets and dataloaders.
train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# ====== Section 4: Model Definition ======
# Define a convolutional neural network model.
class CNN2D(nn.Module):
    def __init__(self, num_classes):
        super(CNN2D, self).__init__()
        self.upsample = nn.Upsample(scale_factor=(1, 2), mode='bilinear', align_corners=False)
        self.conv1 = nn.Conv2d(2, 64, kernel_size=(1, 3), padding=(0, 1))
        self.relu1 = nn.ReLU()
        self.adaptive_pool1 = nn.AdaptiveAvgPool2d((1,64))
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 3), padding=(0, 1))
        self.relu2 = nn.ReLU()
        self.adaptive_pool2 = nn.AdaptiveAvgPool2d((1,32))
        self.adaptive_avg_pool2d = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc1 = nn.Linear(128, 256)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = x.unsqueeze(-2)
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.adaptive_pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.adaptive_pool2(x)
        x = self.adaptive_avg_pool2d(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

# Initialize the CNN model.
model = CNN2D(num_classes=2)

In [ ]:
# ====== Section 5: Training and Evaluation Functions ======
# Define functions for training and evaluating the model.
def train(model, train_loader, num_epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(num_epochs):
        for data, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

In [ ]:
# ====== Section 6: Model Quantization for Deployment ======
# Prepare a dummy input for model inspection and quantization.
dummy_input = torch.randn(1, 2, 128)

# Inspect the model using NNDCT Inspector.
inspector = Inspector(target="DPUCVDX8G_ISA3_C32B6")
inspector.inspect(model, (dummy_input,), output_dir="inspect", image_format=None)

# Perform model quantization.
quantizer = pytorch_nndct.torch_quantizer("calib", model, (dummy_input,))
quantizer.export_quant_config()
quantizer = pytorch_nndct.torch_quantizer("test", model, (dummy_input,))
quant_model = quantizer.quant_model
evaluate(quant_model, test_loader)

# Export the quantized model.
quantizer.export_xmodel(deploy_check=False)